<a href="https://colab.research.google.com/github/hellwalker3/Vocal_separation_with_Demucs_and_Speaker_Conversion_with_so-vits-svc4.0/blob/main/Vocal_separation_with_Demucs_and_Speaker_Conversion_with_RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@title Install RVC
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

!git clone  https://github.com/liujing04/Retrieval-based-Voice-Conversion-WebUI.git
#--depth=1
#https://github.com/fumiama/Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

In [ ]:
#@title Checkout
!git checkout ebd938f

In [ ]:
#@title Download pretrained models
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

In [ ]:
#@title Install demucs and yt-dlp
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs
!pip install yt-dlp

In [ ]:
#@title Rewriting for Direct Inference Without Using a GUI (Bypassing config.py loading)

def replace_config_values(file_name: str):
    with open(file_name, "r") as file:
        content = file.read()

    # Replace the old values with the new ones
    new_content = content.replace(
        "from config import x_pad,x_query,x_center,x_max",
        "x_pad = 3\nx_query = 10\nx_center = 60\nx_max = 65",
    )

    # Overwrite the original file with the modified content
    with open(file_name, "w") as file:
        file.write(new_content)

# Usage example
file_name = "/content/Retrieval-based-Voice-Conversion-WebUI/vc_infer_pipeline.py"
replace_config_values(file_name)

In [ ]:
#@title Please Restart the Runtime to Avoid NumPy Errors

In [ ]:
#@title Required configuration Settings for Model, Song, and YouTube URL
# Set the model folder name and folder path
# The folder must include exclusively the *.pth file, the *.index file and the *.nppy file.
model_folder_name = ""
model_folder_path = f"/content/drive/MyDrive/{model_folder_name}"

# Set the song title and folder path
# The folder should contain the music content
# If this is the first time using this song, it will be downloaded from the provided youtube_url and separated
song_title = ""
song_folder_path = f"/content/drive/MyDrive/{song_title}"

# Set the YouTube URL for the song to be downloaded
youtube_url = ""


# The frequency specified during training
target_frequency = 40000

# Inference Settings
sid = 0                  # Speaker ID
f0_up_key = 0            # Fundamental frequency (F0) shift key
f0_file = None           # File containing fundamental frequency (F0) information
f0_method = "pm"         # Method for extracting the fundamental frequency (F0)
index_rate = 1           # Index rate

In [ ]:
#@title Demucs Vocal Separation
%cd /content/Retrieval-based-Voice-Conversion-WebUI
import os, glob, shutil
from pydub import AudioSegment
from IPython.display import Audio, display
#@title Settings of demucs
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

#@title Useful functions of demucs, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

#@title Utility function to download music
from yt_dlp import YoutubeDL

def download_audio(url, save_path, audio_format="mp3"):
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": save_path,
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": audio_format}],
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def create_directories():
    os.makedirs(song_folder_path, exist_ok=True)
    os.makedirs(os.path.join(song_folder_path, "full"), exist_ok=True)

def download_audio_if_needed():
    save_path = os.path.join(song_folder_path, "full/music")
    if not os.path.exists(save_path + ".mp3"):
        download_audio(youtube_url, save_path)

def separate_audio():
    music_path = os.path.join(song_folder_path, "full")
    if not os.path.exists(music_path + "_sep"):
        separate(music_path, music_path + "_sep")

def get_source_path():
    music_path = os.path.join(song_folder_path, "full")
    for file in glob.glob(f"{os.path.join(music_path + '_sep', 'htdemucs/music')}/**"):
        if file[-10:] == "vocals.mp3":
            return file
    return None

def change_audio_frequency(input_path, target_frequency):
    # Load the input audio file
    audio = AudioSegment.from_file(input_path)

    # Change the frequency of the audio
    audio = audio.set_frame_rate(target_frequency)

    # Save the modified audio to the output path
    output_path = "/content/vocals.wav"
    audio.export(output_path, format="wav")

    return output_path


def copy_file_with_suffix(folder_path, suffix):
    target_file = None
    for file_name in os.listdir(folder_path):
        if file_name.endswith(suffix):
            target_file = file_name
            shutil.copy(os.path.join(folder_path, file_name), file_name)
            break
    if target_file is None:
        raise ValueError(f"File with suffix '{suffix}' not found in the folder.")
    return target_file

def copy_config_and_model_files():
    npy_suffix = ".npy"
    index_suffix = ".index"
    pth_suffix = ".pth"

    npy_file = copy_file_with_suffix(model_folder_path, npy_suffix)
    index_file = copy_file_with_suffix(model_folder_path, index_suffix)
    pth_file = copy_file_with_suffix(model_folder_path, pth_suffix)

    return npy_file, index_file, pth_file


create_directories()
download_audio_if_needed()
separate_audio()

source_path = get_source_path()
source_path = change_audio_frequency(source_path, target_frequency)
npy_file, index_file, pth_file = copy_config_and_model_files()

In [ ]:
#@title  Conversion with RVC
is_half = True
device = 'cuda:0'

from fairseq import checkpoint_utils
from my_utils import load_audio
import traceback
import torch
from infer_pack.models import SynthesizerTrnMs256NSFsid, SynthesizerTrnMs256NSFsid_nono
from vc_infer_pipeline import VC

def get_vc(sid):
    global n_spk,tgt_sr,net_g,vc,cpt
    if(sid==[]):
        global hubert_model
        if (hubert_model != None): # 考虑到轮询，需要加个判断看是否 sid 是由有模型切换到无模型的
            print("clean_empty_cache")
            del net_g, n_spk, vc, hubert_model,tgt_sr#,cpt
            hubert_model = net_g=n_spk=vc=hubert_model=tgt_sr=None
            torch.cuda.empty_cache()
            ###楼下不这么折腾清理不干净
            if_f0 = cpt.get("f0", 1)
            if (if_f0 == 1):
                net_g = SynthesizerTrnMs256NSFsid(*cpt["config"], is_half=is_half)
            else:
                net_g = SynthesizerTrnMs256NSFsid_nono(*cpt["config"])
            del net_g,cpt
            torch.cuda.empty_cache()
            cpt=None
        return {"visible": False, "__type__": "update"}
    person = sid
    print("loading %s"%person)
    cpt = torch.load(person, map_location="cpu")
    tgt_sr = cpt["config"][-1]
    cpt["config"][-3]=cpt["weight"]["emb_g.weight"].shape[0]#n_spk
    if_f0=cpt.get("f0",1)
    if(if_f0==1):
        net_g = SynthesizerTrnMs256NSFsid(*cpt["config"], is_half=is_half)
    else:
        net_g = SynthesizerTrnMs256NSFsid_nono(*cpt["config"])
    del net_g.enc_q
    print(net_g.load_state_dict(cpt["weight"], strict=False))  # 不加这一行清不干净，真奇葩
    net_g.eval().to(device)
    if (is_half):net_g = net_g.half()
    else:net_g = net_g.float()
    vc = VC(tgt_sr, device, is_half)
    n_spk=cpt["config"][-3]
    return {"visible": True,"maximum": n_spk, "__type__": "update"}

hubert_model=None
def load_hubert():
    global hubert_model
    models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task(["hubert_base.pt"],suffix="",)
    hubert_model = models[0]
    hubert_model = hubert_model.to(device)
    if(is_half):hubert_model = hubert_model.half()
    else:hubert_model = hubert_model.float()
    hubert_model.eval()

def vc_single(sid,input_audio,f0_up_key,f0_file,f0_method,file_index,file_big_npy,index_rate):#spk_item, input_audio0, vc_transform0,f0_file,f0method0
    global tgt_sr,net_g,vc,hubert_model
    if input_audio is None:return "You need to upload an audio", None
    f0_up_key = int(f0_up_key)
    try:
        audio=load_audio(input_audio,16000)
        times = [0, 0, 0]
        if(hubert_model==None):load_hubert()
        if_f0 = cpt.get("f0", 1)
        audio_opt=vc.pipeline(hubert_model,net_g,sid,audio,times,f0_up_key,f0_method,file_index,file_big_npy,index_rate,if_f0,f0_file=f0_file)
        print(times)
        return "Success", (tgt_sr, audio_opt)
    except:
        info=traceback.format_exc()
        print(info)
        return info,(None,None)

from scipy.io import wavfile
from pydub import AudioSegment

def save_numpy_as_mp3(audio_array, sample_rate, output_file):
    # Save the audio array as a WAV file
    wavfile.write(output_file, sample_rate, audio_array)

# Add the save_numpy_as_mp3() function here
def run_conversion_and_cleanup(pth_file, source_path, index_file, npy_file):
    global tgt_sr, n_spk,cpt,net_g,vc,hubert_model
    get_vc(pth_file)
    input_audio = source_path
    index = index_file
    npy = npy_file
    info, (tgt_sr, audio_opt) = vc_single(sid, input_audio, f0_up_key, f0_file, f0_method, index, npy, index_rate)

    if audio_opt is not None:
        # Save output as an MP3 file
        output_file = "/content/output_vocals.wav"
        save_numpy_as_mp3(audio_opt, tgt_sr, output_file)
        print(f"Audio saved as {output_file}")

        # Clean up intermediate variables
        
        del n_spk,cpt,tgt_sr,net_g,vc,hubert_model
        torch.cuda.empty_cache()

    else:
        print("Error occurred:", info)

# Run the conversion and cleanup
run_conversion_and_cleanup(pth_file, source_path, index_file, npy_file)

In [ ]:
#@title Music Integration to generate /content/output_music.mp3 (with bgm)
from pydub import AudioSegment
import numpy as np

def detect_silence(audio, threshold=-40, min_silence_len=1000):
    silent_ranges = []
    start = None

    for t in range(len(audio)):
        sample = audio[t]
        if sample.dBFS < threshold:
            if start is None:
                start = t
        else:
            if start is not None:
                end = t
                if end - start >= min_silence_len:
                    silent_ranges.append((start, end))
                start = None

    if start is not None:
        silent_ranges.append((start, len(audio)))

    return silent_ranges

def remove_noise(source_path, output_path, threshold = -40):

  # 音声ファイルをロード
  source_audio = AudioSegment.from_wav(source_path)
  output_audio = AudioSegment.from_wav(output_path)

  # 無音区間を検出
  silent_ranges = detect_silence(source_audio, threshold)

  # 変換後の音声ファイルに無音区間を適用
  for start, end in silent_ranges:
      output_audio = output_audio[:start] + AudioSegment.silent(duration=end-start) + output_audio[end:]

  # 結果を保存
  output_audio.export(output_path, format="wav")

def get_full_music_list():
    music_path = os.path.join(song_folder_path, "full")
    return [file_path for file_path in glob.glob(os.path.join(music_path+'_sep', 'htdemucs/music', '**')) if file_path[-10:] != 'vocals.mp3'] + ["/content/output_vocals.wav"]

def mix_audio_segments(full_music_list):
    audio_segments = [AudioSegment.from_file(audio_file, format=audio_file[-3:]) for audio_file in full_music_list]
    mixed = audio_segments[0]
    for audio_segment in audio_segments[1:]:
        mixed = mixed.overlay(audio_segment)
    return mixed

def export_and_display_audio(mixed):
    integrated_audio_path = "/content/output_music.mp3"
    mixed.export(integrated_audio_path, format='mp3')
    display(Audio(integrated_audio_path, autoplay=True))
    
remove_noise(source_path, "/content/output_vocals.wav")
full_music_list = get_full_music_list()
mixed_audio = mix_audio_segments(full_music_list)
export_and_display_audio(mixed_audio)

In [ ]:
#@title Download generated file
from google.colab import files
files.download("/content/output_music.mp3")